In [1]:
import os
import json
import meshio
import polyscope as ps
import polyscope.imgui as psim
import numpy as np
from collections import Counter


In [2]:

# Global parameters
scale_factor = 1.0
rotation_degrees = [90.0, 0.0, 0.0]  # Rotation around X, Y, Z axes
translation = [0.0, 0.0, 0.0]
bottom_percent = 10.0
top_percent = 0.0
right_percent = 0.0
left_percent = 0.0
bounding_box_results = ""

original_points = None
bottom_4_points = None
top_4_points = None
right_4_points = None
left_4_points = None

# We'll store the transformed global bounding box
mesh_min_corner_t = None
mesh_max_corner_t = None

In [3]:

def load_mesh(file_path):
    """Load a mesh file using meshio and return its points and cells."""
    mesh = meshio.read(file_path)
    return mesh.points, mesh.cells


In [4]:

def extract_surface_faces(cells):
    """Extract boundary faces from tetrahedral or triangular meshes."""
    all_faces = []
    for cell_block in cells:
        if cell_block.type == "tetra":
            for c in cell_block.data:
                faces = [
                    tuple(sorted([c[0], c[1], c[2]])),
                    tuple(sorted([c[0], c[1], c[3]])),
                    tuple(sorted([c[0], c[2], c[3]])),
                    tuple(sorted([c[1], c[2], c[3]])),
                ]
                all_faces.extend(faces)
        elif cell_block.type == "triangle":
            for c in cell_block.data:
                faces = tuple(sorted(c))
                all_faces.append(faces)

    face_count = Counter(all_faces)
    boundary_faces = [f for f, count in face_count.items() if count == 1]
    return np.array(boundary_faces)


In [5]:

def apply_transforms(points):
    """Apply scaling, rotation, and translation to the points."""
    p = points * scale_factor
    rx, ry, rz = np.radians(rotation_degrees)
    Rx = np.array([[1, 0, 0],
                   [0, np.cos(rx), -np.sin(rx)],
                   [0, np.sin(rx), np.cos(rx)]])
    Ry = np.array([[np.cos(ry), 0, np.sin(ry)],
                   [0, 1, 0],
                   [-np.sin(ry), 0, np.cos(ry)]])
    Rz = np.array([[np.cos(rz), -np.sin(rz), 0],
                   [np.sin(rz), np.cos(rz), 0],
                   [0, 0, 1]])
    p = p @ (Rz @ Ry @ Rx).T
    return p + translation


In [6]:

def get_percentage_points(points, bottom_percent, top_percent, right_percent, left_percent):
    """Get bottom and top and right and left percentage of points."""
    sorted_z = np.argsort(points[:, 1])
    sorted_points_z = points[sorted_z]

    sorted_x = np.argsort(points[:, 0])
    sorted_points_x = points[sorted_x]

    total_points = len(points)
    bottom_count = max(1, int((bottom_percent / 100.0) * total_points))
    top_count = max(1, int((top_percent / 100.0) * total_points))

    right_count = max(1, int((right_percent / 100.0) * total_points))
    left_count = max(1, int((left_percent / 100.0) * total_points))

    bottom = sorted_points_z[:bottom_count] if bottom_count > 0 else np.empty((0, 3))
    top = sorted_points_z[-top_count:] if top_count > 0 else np.empty((0, 3))
    right = sorted_points_x[-right_count:] if right_count > 0 else np.empty((0, 3))
    left = sorted_points_x[:left_count] if left_count > 0 else np.empty((0, 3))

    return bottom, top, right, left


In [7]:

def compute_bounding_box(points):
    """Compute the axis-aligned bounding box of given points."""
    if len(points) == 0:
        return None, None
    min_corner = np.min(points, axis=0)
    max_corner = np.max(points, axis=0)
    return min_corner, max_corner


In [8]:

def bounding_box_line_network(min_corner, max_corner):
    """Generate line segments representing the bounding box."""
    x0, y0, z0 = min_corner
    x1, y1, z1 = max_corner
    corners = np.array([
        [x0, y0, z0], [x0, y0, z1], [x0, y1, z0], [x0, y1, z1],
        [x1, y0, z0], [x1, y0, z1], [x1, y1, z0], [x1, y1, z1],
    ])
    edges = np.array([
        [0, 1], [0, 2], [2, 3], [1, 3],
        [4, 5], [4, 6], [5, 7], [6, 7],
        [0, 4], [1, 5], [2, 6], [3, 7]
    ])
    return corners, edges


In [9]:
def update_geometry():
    """Update geometry positions based on transformations."""
    global bottom_4_points, top_4_points, right_4_points, left_4_points

    transformed_points = apply_transforms(original_points)
    ps.get_surface_mesh("Original Mesh").update_vertex_positions(transformed_points)

    # Compute new bottom, top, right, and left points
    bottom_4_points, top_4_points, right_4_points, left_4_points = get_percentage_points(
        transformed_points, bottom_percent, top_percent, right_percent, left_percent
    )

    # Register Bottom Points
    if ps.has_point_cloud("Bottom Points"):
        ps.remove_point_cloud("Bottom Points")
    if len(bottom_4_points) > 0:
        ps.register_point_cloud("Bottom Points", bottom_4_points, color=[0, 0, 1], radius=0.005)  # Blue

    # Register Top Points
    if ps.has_point_cloud("Top Points"):
        ps.remove_point_cloud("Top Points")
    if len(top_4_points) > 0:
        ps.register_point_cloud("Top Points", top_4_points, color=[1, 0, 0], radius=0.005)  # Red

    # Register Right Points
    if ps.has_point_cloud("Right Points"):
        ps.remove_point_cloud("Right Points")
    if len(right_4_points) > 0:
        ps.register_point_cloud("Right Points", right_4_points, color=[0, 1, 0], radius=0.005)  # Green

    # Register Left Points
    if ps.has_point_cloud("Left Points"):
        ps.remove_point_cloud("Left Points")
    if len(left_4_points) > 0:
        ps.register_point_cloud("Left Points", left_4_points, color=[0, 1, 1], radius=0.005)  # Cyan

    # Update bounding boxes and export
    update_boxes()

In [10]:

def update_boxes():
    """Add bounding boxes for top, bottom, right, and left selected points and export to JSON."""
    global bounding_box_results
    global mesh_min_corner_t, mesh_max_corner_t

    results = []
    export_data = {}

    # Utility to compute relative coordinates in [0,1]
    def relative_coords(pt):
        return (pt - mesh_min_corner_t) / (mesh_max_corner_t - mesh_min_corner_t)

    # Function to handle each box
    def handle_box(points, box_name, color):
        if points is not None and len(points) > 0:
            min_corner, max_corner = compute_bounding_box(points)
            if min_corner is None or max_corner is None:
                return
            box_corners, box_edges = bounding_box_line_network(min_corner, max_corner)
            if ps.has_curve_network(f"{box_name} Box"):
                ps.remove_curve_network(f"{box_name} Box")
            ps.register_curve_network(f"{box_name} Box", box_corners, box_edges, color=color, radius=0.005)
            results.append(f"{box_name} Box Min: {min_corner}, Max: {max_corner}")

            rel_min = relative_coords(min_corner)
            rel_max = relative_coords(max_corner)

            # Clip values to [0,1] just in case
            rel_min = np.clip(rel_min, 0, 1)
            rel_max = np.clip(rel_max, 0, 1)

            export_data[f"{box_name.lower()}_box"] = {
                "absolute": [min_corner.tolist(), max_corner.tolist()],
                "relative": [rel_min.tolist(), rel_max.tolist()]
            }

    # Bottom Box
    handle_box(bottom_4_points, "Bottom", [0, 0, 1])

    # Top Box
    handle_box(top_4_points, "Top", [1, 0, 0])

    # Right Box
    handle_box(right_4_points, "Right", [0, 1, 0])

    # Left Box
    handle_box(left_4_points, "Left", [0, 1, 1])

    bounding_box_results = "\n".join(results)

    # Write the exported data to a JSON file
    if export_data:
        with open("export_data.json", "w") as f:
            json.dump(export_data, f, indent=4)

In [11]:

def user_callback():
    """Polyscope UI for user interaction."""
    global scale_factor, rotation_degrees, translation, bottom_percent, top_percent, right_percent, left_percent

    # Input for scale
    changed, scale = psim.InputFloat("Scale", scale_factor, 0.1, 1.0, "%.3f")
    if changed:
        scale_factor = scale
        update_geometry()

    # Input for rotation around X, Y, Z
    for i, axis in enumerate("XYZ"):
        changed, val = psim.InputFloat(f"Rotation {axis} (deg)", rotation_degrees[i], 1.0, 10.0, "%.1f")
        if changed:
            rotation_degrees[i] = val
            update_geometry()

    # Input for translation along X, Y, Z
    for i, axis in enumerate("XYZ"):
        changed, val = psim.InputFloat(f"Translate {axis}", translation[i], 0.1, 1.0, "%.3f")
        if changed:
            translation[i] = val
            update_geometry()

    # Input for bottom and top percentages
    changed, new_bottom_percent = psim.InputFloat("Bottom %", bottom_percent, 0.0, 100.0, "%.1f")
    if changed:
        bottom_percent = max(0.0, min(100.0, new_bottom_percent))
        update_geometry()

    changed, new_top_percent = psim.InputFloat("Top %", top_percent, 0.0, 100.0, "%.1f")
    if changed:
        top_percent = max(0.0, min(100.0, new_top_percent))
        update_geometry()

    # Input for right and left percentages
    changed, new_right_percent = psim.InputFloat("Right %", right_percent, 0.0, 100.0, "%.1f")
    if changed:
        right_percent = max(0.0, min(100.0, new_right_percent))
        update_geometry()

    changed, new_left_percent = psim.InputFloat("Left %", left_percent, 0.0, 100.0, "%.1f")
    if changed:
        left_percent = max(0.0, min(100.0, new_left_percent))
        update_geometry()

    # Reset button
    if psim.Button("Reset"):
        scale_factor = 1.0
        rotation_degrees = [0.0, 0.0, 0.0]
        translation = [0.0, 0.0, 0.0]
        bottom_percent = 10.0
        top_percent = 0.0
        right_percent = 0.0
        left_percent = 0.0
        update_geometry()

    # Display bounding box results
    psim.Text(bounding_box_results)


In [12]:

def main():
    global original_points, mesh_min_corner_t, mesh_max_corner_t

    file_path = os.path.join(os.getcwd(), "../../meshes/plat_tetra.msh")
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Mesh file not found: {file_path}")

    points, cells = load_mesh(file_path)
    original_points = points

    # Compute transformed points once at startup so we have a global bounding box after transformations
    transformed_mesh_points = apply_transforms(original_points)
    mesh_min_corner_t, mesh_max_corner_t = compute_bounding_box(transformed_mesh_points)

    ps.init()
    ps.set_navigation_style("turntable")

    ps.register_surface_mesh("Original Mesh", transformed_mesh_points, extract_surface_faces(cells), smooth_shade=True)
    # Initially register with a single point; this will be replaced on update
    ps.register_point_cloud("Bottom Points", np.zeros((1, 3)), color=[0, 0, 1], radius=0.005)
    ps.register_point_cloud("Top Points", np.zeros((1, 3)), color=[1, 0, 0], radius=0.005)
    ps.register_point_cloud("Right Points", np.zeros((1, 3)), color=[1, 0, 0], radius=0.005)
    ps.register_point_cloud("Left Points", np.zeros((1, 3)), color=[1, 0, 0], radius=0.005)

    update_geometry()

    ps.set_user_callback(user_callback)
    ps.show()

if __name__ == "__main__":
    main()
